In [1]:
import sys
from pathlib import Path

project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

import os
os.chdir(project_root)

In [2]:
import torch
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

from src.data.pipeline import get_datasets
from src.models.transformer_model import StockTransformer
from src.evaluation.metrics import calculate_metrics
from src.utils.config import load_config

config = load_config()

print("Зареждане на test данни...")
_, _, test_dataset, feature_columns = get_datasets(config)

test_loader = DataLoader(
    test_dataset,
    batch_size=config.training.batch_size,
    shuffle=False,
    num_workers=0,
)

print(f"Test samples: {len(test_dataset)}")
print(f"Features: {len(feature_columns)}")

In [3]:
checkpoint_path = Path(config.paths.models_dir) / "best_model.pt"
checkpoint = torch.load(checkpoint_path, map_location='cpu')

print(f"Checkpoint: epoch {checkpoint['epoch'] + 1}, val loss: {checkpoint['score']:.6f}")

state_dict = checkpoint['model_state_dict']
input_dim_checkpoint = state_dict['input_projection.weight'].shape[1]
input_dim_current = len(feature_columns)

d_model = state_dict['input_projection.weight'].shape[0]
n_layers = len([k for k in state_dict.keys() if 'encoder.layers' in k and 'self_attention.w_q.weight' in k])
n_heads = config.model.n_heads
d_ff = state_dict['encoder.layers.0.feed_forward.linear1.weight'].shape[0]

print(f"\nМодел параметри от checkpoint:")
print(f"  input_dim: {input_dim_checkpoint}")
print(f"  d_model: {d_model}")
print(f"  n_layers: {n_layers}")
print(f"  d_ff: {d_ff}")
print(f"\nТекущи данни:")
print(f"  input_dim: {input_dim_current}")

if input_dim_checkpoint != input_dim_current:
    print(f"\nВНИМАНИЕ: Разлика в features!")
    print(f"Създавам модел с текущите features ({input_dim_current})...")
    
    model = StockTransformer(
        input_dim=input_dim_current,
        d_model=d_model,
        n_heads=n_heads,
        n_layers=n_layers,
        d_ff=d_ff,
        dropout=config.model.dropout,
        activation=config.model.activation,
        prediction_horizon=config.data.prediction_horizon,
    )
    
    model_dict = model.state_dict()
    compatible_dict = {k: v for k, v in state_dict.items() 
                      if k in model_dict and model_dict[k].shape == v.shape}
    
    model_dict.update(compatible_dict)
    model.load_state_dict(model_dict, strict=False)
    print(f"Заредени {len(compatible_dict)}/{len(state_dict)} параметъра")
    print("ВНИМАНИЕ: input_projection е нов (необучен) - резултатите може да са по-слаби!")
else:
    model = StockTransformer(
        input_dim=input_dim_checkpoint,
        d_model=d_model,
        n_heads=n_heads,
        n_layers=n_layers,
        d_ff=d_ff,
        dropout=config.model.dropout,
        activation=config.model.activation,
        prediction_horizon=config.data.prediction_horizon,
    )
    model.load_state_dict(state_dict)
    print(f"\nМодел зареден успешно!")

model.eval()

In [ ]:
all_predictions = []
all_targets = []

print("Тестване на модела...")
with torch.no_grad():
    for batch_x, batch_y in test_loader:
        predictions = model(batch_x)
        
        if predictions.dim() == 1:
            predictions = predictions.unsqueeze(1)
        if batch_y.dim() == 1:
            batch_y = batch_y.unsqueeze(1)
        
        all_predictions.append(predictions)
        all_targets.append(batch_y)

predictions = torch.cat(all_predictions, dim=0)
targets = torch.cat(all_targets, dim=0)

metrics = calculate_metrics(predictions, targets)

print("\n" + "="*60)
print("РЕЗУЛТАТИ НА TEST SET:")
print("="*60)
for metric_name, value in metrics.items():
    if metric_name == "mape":
        print(f"{metric_name.upper()}: {value:.2f}%")
    elif metric_name == "directional_accuracy":
        print(f"{metric_name.upper()}: {value*100:.2f}%")
    else:
        print(f"{metric_name.upper()}: {value:.6f}")
print("="*60)

In [ ]:
predictions_np = predictions.numpy().flatten()
targets_np = targets.numpy().flatten()

plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
plt.plot(targets_np[:200], label="Actual", alpha=0.7, linewidth=1.5)
plt.plot(predictions_np[:200], label="Predicted", alpha=0.7, linewidth=1.5)
plt.xlabel("Time Step")
plt.ylabel("Normalized Price")
plt.title("Predictions vs Actual (First 200 samples)")
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 2)
plt.scatter(targets_np, predictions_np, alpha=0.3, s=10)
min_val = min(np.min(targets_np), np.min(predictions_np))
max_val = max(np.max(targets_np), np.max(predictions_np))
plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label="Perfect Prediction")
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Scatter Plot")
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 3)
residuals = targets_np - predictions_np
plt.plot(residuals[:200], alpha=0.7)
plt.axhline(y=0, color='r', linestyle='--', linewidth=1)
plt.xlabel("Time Step")
plt.ylabel("Residual")
plt.title("Residuals (First 200 samples)")
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 4)
plt.hist(residuals, bins=50, alpha=0.7, edgecolor='black')
plt.xlabel("Residual")
plt.ylabel("Frequency")
plt.title("Residuals Distribution")
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nСтатистики на residuals:")
print(f"  Mean: {np.mean(residuals):.6f}")
print(f"  Std: {np.std(residuals):.6f}")
print(f"  Min: {np.min(residuals):.6f}")
print(f"  Max: {np.max(residuals):.6f}")